In [3]:
import numpy as np
import pandas as pd
import os
import pickle
import torch

from scipy.sparse import load_npz, issparse
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from lightgbm import LGBMClassifier
from sklearn.metrics import accuracy_score, roc_auc_score, confusion_matrix, classification_report


In [4]:

# --- 1. Load Final Feature Matrix and Target ---

# Define the paths where the feature engineering step saved the data
base_path = '/home/vikhil/GROUP_1-INFOSYS/Member_Vikhil/Datasets/'
save_path_X = os.path.join(base_path, 'X_final_features.npz')
save_path_y = os.path.join(base_path, 'y_target.pkl')

print(f"Loading features from: {save_path_X}")
print(f"Loading target from: {save_path_y}")

# Load the sparse feature matrix (X)
X_final = load_npz(save_path_X)

# Load the target vector (y)
with open(save_path_y, 'rb') as f:
    y = pickle.load(f)

# Optional: Set PyTorch device context (for general GPU awareness)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device for PyTorch context: {device}")
print(f"Loaded Feature Matrix Shape: {X_final.shape}")





Loading features from: /home/vikhil/GROUP_1-INFOSYS/Member_Vikhil/Datasets/X_final_features.npz
Loading target from: /home/vikhil/GROUP_1-INFOSYS/Member_Vikhil/Datasets/y_target.pkl
Using device for PyTorch context: cuda
Loaded Feature Matrix Shape: (17880, 8634)


In [5]:
# --- 2. Data Splitting and Imbalance Calculation ---

# Split data into training and testing sets (70/30)
# Stratify=y ensures both sets have the same proportion of fraudulent cases
X_train, X_test, y_train, y_test = train_test_split(
    X_final, y, test_size=0.3, random_state=42, stratify=y
)

print("\n--- Data Split Check ---")
print(f"Training set size: {X_train.shape[0]} samples")
print(f"Testing set size: {X_test.shape[0]} samples")

# Calculate the imbalance ratio for LightGBM's scale_pos_weight
total_non_fraud = np.sum(y_train == 0)
total_fraud = np.sum(y_train == 1)
scale_pos_weight = total_non_fraud / total_fraud

print(f"Imbalance Ratio (Non-Fraud / Fraud): {scale_pos_weight:.2f}")


--- Data Split Check ---
Training set size: 12516 samples
Testing set size: 5364 samples
Imbalance Ratio (Non-Fraud / Fraud): 19.65


In [6]:
# ----------------------------------------------------
## Step 3: Model Training
# ----------------------------------------------------

# 3.1 Baseline Model: Logistic Regression
print("\n--- Training Logistic Regression (Baseline) ---")
lr_model = LogisticRegression(
    solver='liblinear',        # Good for smaller datasets and L1/L2 penalties
    class_weight='balanced',   # Automatically weighs classes inverse to their frequency
    random_state=42,
    max_iter=500
)
lr_model.fit(X_train, y_train)
print("Logistic Regression Training Complete.")

# 3.2 Ensemble Model: LightGBM
print("\n--- Training LightGBM Classifier ---")
lgbm_model = LGBMClassifier(
    objective='binary',
    metric='auc',
    n_estimators=1000,
    learning_rate=0.05,
    scale_pos_weight=scale_pos_weight, # Use the calculated ratio to handle imbalance
    random_state=42,
    n_jobs=-1,
    verbose=-1
)
lgbm_model.fit(X_train, y_train)
print("LightGBM Training Complete.")


--- Training Logistic Regression (Baseline) ---
Logistic Regression Training Complete.

--- Training LightGBM Classifier ---
LightGBM Training Complete.


In [7]:

# ----------------------------------------------------
## Step 4: Evaluation
# ----------------------------------------------------

print("\n=====================================================")
print("             MODEL EVALUATION (LightGBM)             ")
print("=====================================================")

# Predict probabilities and labels on the test set
y_pred_proba = lgbm_model.predict_proba(X_test)[:, 1]
y_pred = lgbm_model.predict(X_test)

# 4.1 ROC AUC Score
roc_auc = roc_auc_score(y_test, y_pred_proba)
print(f"ROC AUC Score: {roc_auc:.4f} (Measures class separation)")

# 4.2 Accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.4f}")

# 4.3 Classification Report (Precision, Recall, F1)
print("\nClassification Report (Focus on Class 1: Fraudulent):")
print(classification_report(y_test, y_pred, target_names=['Non-Fraud (0)', 'Fraud (1)']))

# 4.4 Confusion Matrix
conf_matrix = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:")
print(conf_matrix)


             MODEL EVALUATION (LightGBM)             
ROC AUC Score: 0.9935 (Measures class separation)


/home/vikhil/miniconda3/envs/vikhil/lib/python3.10/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


Accuracy: 0.9903

Classification Report (Focus on Class 1: Fraudulent):
               precision    recall  f1-score   support

Non-Fraud (0)       0.99      1.00      0.99      5104
    Fraud (1)       0.96      0.83      0.89       260

     accuracy                           0.99      5364
    macro avg       0.98      0.91      0.94      5364
 weighted avg       0.99      0.99      0.99      5364

Confusion Matrix:
[[5096    8]
 [  44  216]]


In [8]:
import pickle
import os

# --- Define Paths (Use the same base path as before) ---
base_path = '/home/vikhil/GROUP_1-INFOSYS/Member_Vikhil/Datasets/'

# Define specific save paths for the models
save_path_lgbm_model = os.path.join(base_path, 'lgbm_model.pkl') 
save_path_lr_model = os.path.join(base_path, 'lr_model_baseline.pkl')

print("\n--- Saving Trained Models ---")

# 1. Save the LightGBM Model (Production Model)
try:
    with open(save_path_lgbm_model, 'wb') as f:
        pickle.dump(lgbm_model, f)
    print(f"✅ LightGBM Model Saved to: {save_path_lgbm_model}")
except NameError:
    print("❌ ERROR: 'lgbm_model' is not defined. Ensure the training cell was run successfully.")
    
# 2. Save the Logistic Regression Model (Baseline)
try:
    with open(save_path_lr_model, 'wb') as f:
        pickle.dump(lr_model, f)
    print(f"✅ Logistic Regression Model Saved to: {save_path_lr_model}")
except NameError:
    print("❌ ERROR: 'lr_model' is not defined. Ensure the training cell was run successfully.")


--- Saving Trained Models ---
✅ LightGBM Model Saved to: /home/vikhil/GROUP_1-INFOSYS/Member_Vikhil/Datasets/lgbm_model.pkl
✅ Logistic Regression Model Saved to: /home/vikhil/GROUP_1-INFOSYS/Member_Vikhil/Datasets/lr_model_baseline.pkl


In [18]:
import pandas as pd
import numpy as np
import os
import pickle
from scipy.sparse import hstack, csr_matrix
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from torchtext.data.utils import get_tokenizer
import string
import nltk

# ----------------------------
# CONFIGURATION (Paths)
# ----------------------------
base_path = '/home/vikhil/GROUP_1-INFOSYS/Member_Vikhil/Datasets/'
load_path_encoders = os.path.join(base_path, 'ml_encoders.pkl')
load_path_lgbm_model = os.path.join(base_path, 'lgbm_model.pkl')

# ----------------------------
# LOAD MODELS AND ENCODERS
# ----------------------------
try:
    with open(load_path_encoders, 'rb') as f:
        loaded_tools = pickle.load(f)

    tfidf_title_vectorizer = loaded_tools['tfidf_title']
    tfidf_desc_vectorizer = loaded_tools['tfidf_desc']
    tfidf_profile_vectorizer = loaded_tools['tfidf_profile']
    ohe_categorical = loaded_tools['ohe_categorical']

    with open(load_path_lgbm_model, 'rb') as f:
        lgbm_model = pickle.load(f)

    lemmatizer = WordNetLemmatizer()
    stop_words = set(stopwords.words('english'))
    tokenizer = get_tokenizer("basic_english")

    print("✅ All models and encoders loaded successfully.")
except Exception as e:
    print(f"❌ ERROR: Failed to load model or encoders. Check paths/files: {e}")
    exit()

# ----------------------------
# HELPER FUNCTIONS
# ----------------------------
def clean_text(text):
    if text is None or str(text).lower() == 'nan':
        return ""
    text = BeautifulSoup(str(text), "html.parser").get_text()
    text = text.lower().translate(str.maketrans('', '', string.punctuation))
    tokens = [lemmatizer.lemmatize(t) for t in tokenizer(text) if t.isalpha() and t not in stop_words]
    return ' '.join(tokens)

def segment_job_text(raw_text):
    lines = [l.strip() for l in raw_text.split("\n") if l.strip()]
    title = lines[0] if len(lines) > 0 else "Unknown"
    company_profile = " ".join(lines[:1])
    description = " ".join(lines[1:3])
    requirements = " ".join(lines[3:4])
    benefits = " ".join(lines[4:])
    location = "Unknown"
    return title, location, company_profile, description, requirements, benefits

def preprocess_and_predict(job_data: dict, model):
    df_single = pd.DataFrame([job_data])
    
    # --- Clean text ---
    df_single['cleaned_title'] = df_single['title'].apply(clean_text)
    df_single['cleaned_description'] = df_single['description'].apply(clean_text)
    df_single['cleaned_company_profile'] = df_single['company_profile'].apply(clean_text)

    # --- Derived features ---
    df_single['title_len'] = df_single['title'].apply(len)
    df_single['desc_len'] = df_single['description'].apply(len)
    df_single['profile_len'] = df_single['company_profile'].apply(len)
    df_single['is_senior_role'] = df_single['required_experience'].str.lower().apply(
        lambda x: 1 if any(k in x for k in ['senior','executive','director','manager']) else 0
    )
    df_single['has_salary'] = df_single['salary_range'].apply(lambda x: 0 if x in ['Unknown',''] else 1)

    # --- Text features ---
    X_text = hstack([
        tfidf_title_vectorizer.transform(df_single['cleaned_title']),
        tfidf_desc_vectorizer.transform(df_single['cleaned_description']),
        tfidf_profile_vectorizer.transform(df_single['cleaned_company_profile'])
    ])

    # --- Ensure all categorical columns exist ---
    cat_cols = ['country_code','employment_type','required_experience','required_education',
                'industry','function','department']
    for col in cat_cols:
        if col not in df_single.columns:
            df_single[col] = 'Unknown'
    
    X_cat = ohe_categorical.transform(df_single[cat_cols])

    # --- Numerical features used in training ONLY ---
    num_cols = ['telecommuting','has_company_logo','has_questions','title_len','desc_len','profile_len','is_senior_role','has_salary']
    X_num = csr_matrix(df_single[num_cols].values)

    # --- Stack all features ---
    X_final = hstack([X_text, X_cat, X_num])

    # --- Prediction ---
    pred = model.predict(X_final)[0]
    prob = model.predict_proba(X_final)[:,1][0]  # probability of class 1 (FRAUD)
    
    return pred, prob

# ----------------------------
# MAIN INTERACTIVE INPUT
# ----------------------------
print("\n--- Enter the Raw Job Description ---")
print("Paste the entire text (including Qualifications/Benefits). Press ENTER twice to finish.\n")
raw_lines = []
while True:
    try:
        line = input()
        if not line:
            break
        raw_lines.append(line)
    except EOFError:
        break

raw_input_text = "\n".join(raw_lines)
title, location, company_profile, description, requirements, benefits = segment_job_text(raw_input_text)

user_job_input = {
    'job_id': 100003,
    'title': title,
    'location': location,
    'department': 'Unknown',
    'salary_range': 'Unknown',
    'company_profile': company_profile,
    'description': description,
    'requirements': requirements,
    'benefits': benefits,
    'telecommuting': 0,
    'has_company_logo': 0,
    'has_questions': 0,
    'employment_type': 'Unknown',
    'required_experience': 'Unknown',
    'required_education': 'Unknown',
    'industry': 'Unknown',
    'function': 'Unknown'
}

pred, prob = preprocess_and_predict(user_job_input, model=lgbm_model)
status = "FRAUDULENT" if pred==1 else "LEGITIMATE"
confidence = prob if status=="FRAUDULENT" else 1-prob

print("\n=====================================================")
print("          JOB CLASSIFICATION RISK REPORT 🎯          ")
print("=====================================================")
print(f"Job Title: {user_job_input['title']}")
print(f"Company Profile Length: {len(user_job_input['company_profile'])} characters")
print("-"*50)
print(f"Model Prediction: {status}")
print(f"Confidence (P({status})): {confidence:.4f}")
print("-"*50)

# --- Application advice ---
if status=="LEGITIMATE":
    print("🟢 High confidence real job." if confidence>0.95 else "🟡 Low-risk real job. Verify details.")
else:
    print("🔴 HIGH-RISK FRAUD ALERT." if prob>0.90 else "🟠 Medium-risk fraud. Manual review recommended.")


✅ All models and encoders loaded successfully.

--- Enter the Raw Job Description ---
Paste the entire text (including Qualifications/Benefits). Press ENTER twice to finish.




          JOB CLASSIFICATION RISK REPORT 🎯          
Job Title: Job description Ready to shape the future of work?   At Genpact, we don't just adapt to change we drive it. AI and digital innovation are redefining industries and were leading the charge. Genpacts AI Gigafactory, our industry-first accelerator, is an example of how were scaling advanced technology solutions to help global enterprises work smarter, grow faster, and transform at scale. From large-scale models to agentic AI, our breakthrough solutions tackle companies most complex challenges.   If you thrive in a fast-moving, tech-driven environment, love solving real-world problems, and want to be part of a team that's shaping the future, this is your moment.   Genpact (NYSE: G) is an advanced technology services and solutions company that delivers lasting value for leading enterprises globally. Through our deep business knowledge, operational excellence, and cutting-edge solutions we help companies across industries get a

/home/vikhil/miniconda3/envs/vikhil/lib/python3.10/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/home/vikhil/miniconda3/envs/vikhil/lib/python3.10/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
